In [163]:
import pandas as pd
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx
from torch_geometric.nn import GCNConv

import networkx as nx

In [164]:
features_friends = pd.read_csv("../data/FINAL_FEATURES_FRIENDS.tsv", sep="\t")
target_traintest = pd.read_csv("../data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep="\t")
sequences_matrix = pd.read_csv("../data/FINAL_SEQUENCES_MATRIX.tsv", sep="\t")
targets_traintest = pd.read_csv("../data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep="\t")

In [165]:
targets_train=targets_traintest.loc[targets_traintest.TARGET!='test']

temp_df = features_friends.merge(target_traintest, on='CLIENT_ID', how = 'left')
temp_df = temp_df[:1000]

In [166]:
# temp_df = temp_df[temp_df['TARGET'] != 'test']

In [167]:
edge_attrs_names = temp_df.columns.drop(['CLIENT_ID','FRIEND_ID',
       'RETRO_DT', 'TARGET'])
edge_attrs_val=temp_df[edge_attrs_names]

In [168]:
temp_df['cluster'] = [str(int(x % 2 == 0)) for x in range(len(temp_df))]

In [170]:
G = nx.from_pandas_edgelist(temp_df, 'CLIENT_ID', 'FRIEND_ID')

In [ ]:
G.edges

In [ ]:
temp_df_features = temp_df.drop(['CLIENT_ID','TARGET','RETRO_DT', 'FRIEND_ID', 'cluster'], axis = 1)

In [ ]:
def get_edge_index(data, target):
    node_features_list=list(data.values)
    node_features=torch.tensor(node_features_list)
    node_labels=torch.tensor(target)
    edges_list=edges.values.tolist()
    edge_index01=torch.tensor(edges_list, dtype = torch.long).T
    edge_index02=torch.zeros(edge_index01.shape, dtype = torch.long)#.T
    edge_index02[0,:]=edge_index01[1,:]
    edge_index02[1,:]=edge_index01[0,:]
    edge_index0=torch.cat((edge_index01,edge_index02),axis=1)
    return edge_index0

In [ ]:
edge_indexis = get_edge_index(temp_df_features, temp_df.TARGET.values)

In [ ]:
temp_df_features.values

In [ ]:
g = Data(x = X,
         y = temp_df.TARGET)

In [ ]:
X = torch.from_numpy(temp_df_features.values).to(torch.long)
Y = torch.from_numpy(temp_df.TARGET.values).to(torch.long)

In [ ]:
X = torch.LongTensor(X)

In [ ]:
X

In [ ]:
class SocialGNN(torch.nn.Module):
    def __init__(self,num_of_feat,f):
        super(SocialGNN, self).__init__()
        self.conv1 = GCNConv(num_of_feat, f)
        self.conv2 = GCNConv(f, 2)

    def forward(self, data):
        x = data.x.float()
        edge_index =  data.edge_index
        x = self.conv1(x=x, edge_index=edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
num_of_feat = g.num_node_features

In [ ]:
num_of_feat

In [ ]:
model = SocialGNN(num_of_feat, 16)

In [ ]:
model.forward(g)

In [ ]:
tmp = g.x.float()

In [ ]:
g.edge_index